In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [3]:
def simple_get(url, load_time=1):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    driver = webdriver.Chrome(executable_path=r"C:\Users\usuario\SportComparator\chromedriver.exe")
    driver.implicitly_wait(5)
    driver.get(url)
    driver.implicitly_wait(load_time)
     
    return driver.page_source

In [4]:
def barrabes_get_page_num(raw_html):
    
    html = BeautifulSoup(raw_html, 'lxml')
    
    pag = html.find("div", {"id": "txtPaginacion"})
    
    return pag.find_all('b')[2].text

In [5]:
def barrabes_extract_info(raw_html, category, family, section):
    
    html = BeautifulSoup(raw_html, 'lxml')
    names = []
    brand = []
    url = []
    price = []
    old_price = []
    sub = []
    price_tmp = 0
    
    for box in html.findAll('div', {'class': 'boxProducto'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
            
    for box in html.findAll('div', {'class': 'boxProductof'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
            
    for box in html.findAll('div', {'class': 'boxProductoSki'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
            
    for box in html.findAll('div', {'class': 'boxProductoSkif'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
    
    df = pd.DataFrame(list(zip(names, brand, price, old_price, url)))
    df.columns = ['Name', 'Brand', 'Price', 'Old Price', 'Url']
    
    df['Category'] = category
    df['Family'] = family
    df['Section'] = section
    
    return df

In [6]:
def barrabes_get_family_items(dictionary, category, section):  
    
    df = pd.DataFrame()
    
    for family,url in dictionary.items():

        print(family)
        print(url)
        raw_html = simple_get(url, 7)
        num_pag = barrabes_get_page_num(raw_html)
        
        df_tmp = pd.DataFrame()

        for i in range(int(num_pag)):   
            raw_html = simple_get(url + str(i+1), 7)
            df_tmp = barrabes_extract_info(raw_html, category, family, section)
            df = df.append(df_tmp)
        
    return df
        

In [7]:
from dictionaries_men import chaquetas, chalecos, pantalones, camisetas, ropa_interior, guantes_y_manoplas
from dictionaries_men import gorros_y_tubulares, barrancos, calcetines, calzado, lifestyle

df = pd.DataFrame()

df_tmp = barrabes_get_family_items(chaquetas, "Caquetas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(chalecos, "Chalecos", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(pantalones, "Pantalones", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(camisetas, "Camisetas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(ropa_interior, "Ropa interior", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(guantes_y_manoplas, "Guantes y manoplas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gorros_y_tubulares, "Gorros y tubulares", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(barrancos, "Barrancos", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calcetines, "Calcetines", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calzado, "Calzado", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(lifestyle, "Lifestyle", "Hombre")
df = df.append(df_tmp)

df = df.reset_index(drop=True)



Impermeables
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-impermeables/dep-210010100-d?pag=
Softshell
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-softshell/dep-210010200-d?pag=
Pluma
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-pluma/dep-210010300-d?pag=
Fibra
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-fibra/dep-210010400-d?pag=
Polar Strech
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-polar-stretch/dep-210010500-d?pag=
Forros Polares
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-forros-polares/dep-210010600-d?pag=
Aeróbicas
http://www.barrabes.com/ropa-montaña-hombre/chaquetas-aerobicas/dep-210010700-d?pag=
Softshell
http://www.barrabes.com/ropa-montaña-hombre/chalecos-softshell/dep-210020100-d?pag=
Polares
http://www.barrabes.com/ropa-montaña-hombre/chalecos-polares/dep-210020200-d?pag=
Con relleno
http://www.barrabes.com/ropa-montaña-hombre/chalecos-con-relleno/dep-210020300-d?pag=
Aeróbicos
http://www.barrabes.com/ropa-montaña

In [8]:
from dictionaries_women import chaquetas, chalecos, pantalones, camisetas, ropa_interior, guantes_y_manoplas
from dictionaries_women import gorros_y_tubulares, barrancos, calcetines, calzado, lifestyle

df_tmp = barrabes_get_family_items(chaquetas, "Caquetas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(chalecos, "Chalecos", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(pantalones, "Pantalones", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(camisetas, "Camisetas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(ropa_interior, "Ropa interior", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(guantes_y_manoplas, "Guantes y manoplas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gorros_y_tubulares, "Gorros y tubulares", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(barrancos, "Barrancos", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calcetines, "Calcetines", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calzado, "Calzado", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(lifestyle, "Lifestyle", "Mujer")
df = df.append(df_tmp)

df = df.reset_index(drop=True)


Impermeables
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-impermeables/dep-220010100-d?pag=
Softshell
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-softshell/dep-220010200-d?pag=
Pluma
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-pluma/dep-220010300-d?pag=
Fibra
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-fibra/dep-220010400-d?pag=
Polar Strech
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-polar-stretch/dep-220010500-d?pag=
Forros Polares
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-forros-polares/dep-220010600-d?pag=
Aeróbicas
http://www.barrabes.com/ropa-montaña-mujer/chaquetas-aerobicas/dep-220010700-d?pag=
Softshell
http://www.barrabes.com/ropa-montaña-mujer/chalecos-softshell/dep-220020100-d?pag=
Polares
http://www.barrabes.com/ropa-montaña-mujer/chalecos-polares/dep-220020200-d?pag=
Con relleno
http://www.barrabes.com/ropa-montaña-mujer/chalecos-con-relleno/dep-220020300-d?pag=
Aeróbicos
http://www.barrabes.com/ropa-montaña-mujer/cha

In [9]:
from dictionaries_unisex import relojes, gafas, accesorios_y_limpieza, capas_y_ponchos, monos

df_tmp = barrabes_get_family_items(relojes, "Relojes", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gafas, "Gafas", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(accesorios_y_limpieza, "Accesorios y limpieza", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(capas_y_ponchos, "Capas y ponchos", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(monos, "Monos", "Unisex")
df = df.append(df_tmp)

df = df.reset_index(drop=True)

Relojes de Montaña
http://www.barrabes.com/electronica-orientacion/relojes-montaña-relojes-montaña/dep-530010100-d?pag=
Accesorios
http://www.barrabes.com/electronica-orientacion/relojes-montaña-accesorios/dep-530010200-d?pag=
Fotocromáticas
http://www.barrabes.com/optica/gafas-adulto-fotocromaticas/dep-520010101-d?pag=
Factor 3
http://www.barrabes.com/optica/gafas-adulto-factor-3/dep-520010102-d?pag=
Factor 4
http://www.barrabes.com/optica/gafas-adulto-factor-4/dep-520010103-d?pag=
Cinturones
http://www.barrabes.com/ropa-montaña-hombre/accesorios-limpieza-cinturones/dep-210120100-d?pag=
Mantenimiento
http://www.barrabes.com/ropa-montaña-hombre/accesorios-limpieza-mantenimiento/dep-210120200-d?pag=
Accesorios Running
http://www.barrabes.com/ropa-montaña-hombre/accesorios-limpieza-accesorios-running/dep-210120300-d?pag=
Capas y Ponchos
http://www.barrabes.com/ropa-montaña-hombre/capas-ponchos/dep-210090000-d?pag=
Expedición
http://www.barrabes.com/ropa-montaña-hombre/monos-expedicion/de

In [10]:
from dictionaries_esqui import esqui_de_montaña_y_freeride, esqui_alpino_y_freestyle, esqui_de_fondo, snowboard
from dictionaries_esqui import raquetas, cascos_y_protecciones, seguridad, accesorios_y_mantenimiento

df_tmp = barrabes_get_family_items(esqui_de_montaña_y_freeride, "Esqui de montaña y freeride", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(esqui_alpino_y_freestyle, "Esqui alpino y freestyle", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(esqui_de_fondo, "Esqui de fondo", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(snowboard, "Snowboard", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(raquetas, "Raquetas", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(cascos_y_protecciones, "Cascos y protecciones", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(seguridad, "Seguridad", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(accesorios_y_mantenimiento, "Accesorios y mantenimiento", "Esqui")
df = df.append(df_tmp)

df = df.reset_index(drop=True)

Tablas
http://www.barrabes.com/esqui-raquetas/esqui-montaña-freeride-tablas/edep-800030100-d?pag=
Fijaciones
http://www.barrabes.com/esqui-raquetas/esqui-montaña-freeride-fijaciones/dep-800030200-d?pag=
Botas
http://www.barrabes.com/esqui-raquetas/esqui-montaña-freeride-botas/dep-800030300-d?pag=
Pieles de foca
http://www.barrabes.com/esqui-raquetas/esqui-montaña-freeride-pieles-foca/dep-800030400-d?pag=
Bastones
http://www.barrabes.com/esqui-raquetas/esqui-montaña-freeride-bastones/dep-800030500-d?pag=
Tablas
http://www.barrabes.com/esqui-raquetas/esqui-alpino-freestyle-tablas/edep-800010100-d?pag=
Fijaciones
http://www.barrabes.com/esqui-raquetas/esqui-alpino-freestyle-fijaciones/dep-800010300-d?pag=
Botas
http://www.barrabes.com/esqui-raquetas/esqui-alpino-freestyle-botas/dep-800010200-d?pag=
Bastones
http://www.barrabes.com/esqui-raquetas/esqui-alpino-freestyle-bastones/dep-800010400-d?pag=
Tablas
http://www.barrabes.com/esqui-raquetas/esqui-fondo-tablas/edep-800040100-d?pag=
Fijac

In [11]:
print(df.shape)

(7978, 8)


In [12]:
df.to_csv('BarrabesProducts.csv', sep='\t', encoding='utf-8')